In [1]:
print("Installing Libraries")

!pip install pypdfium2 langchain langchain-google-genai langchain-huggingface faiss-cpu langchain-community

print("\n All packages are installed (Hybrid Ready!)")

Installing Libraries
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 2.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of langchain-google-genai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain-huggingface to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 105.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 70.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━


 All packages are installed (Hybrid Ready!)


In [2]:
# --- STEP 2:PDF Uploading ---
from google.colab import files
import os

if 'pdf_name' not in locals():
  print("Select 'syllabus.pdf' file and upload it...")
  uploaded = files.upload()
  if not uploaded:
    print("\n File was not uploaded app stopped.")
    raise ValueError("File not uploaded!")
  else:
    pdf_name = next(iter(uploaded))
    print(f"\n Super! '{pdf_name}' file was upload.")
else:
  print(f"\n'{pdf_name}' file already present. i will use that.")

Select 'syllabus.pdf' file and upload it...


Saving syllabus.pdf to syllabus.pdf

 Super! 'syllabus.pdf' file was upload.


In [3]:
# --- STEP 3:  Google AI Studio Key setup ---
import getpass

if 'google_api_key' not in locals():
  print("Paste your Google API key here")
  google_api_key = getpass.getpass()
  if not google_api_key.startswith("AIzaSy"):
    print("\n It doesnt look like API key it start like these 'AIzaSy...'.")
    raise ValueError("Invalid Google API Key")
  else:
    print("\n Google API Key received! (Got the Google Key!)")
else:
  print("\n Google API Key already got.")

Paste your Google API key here
··········

 Google API Key received! (Got the Google Key!)


In [6]:
# --- STEP 4: ChatBot Setup (v19 - The FINAL HYBRID) ---

from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
import os, traceback

os.environ["GOOGLE_API_KEY"] = google_api_key
pdf_path = pdf_name

try:
    print(f"'{pdf_path}' Reading...")


    loader = PyPDFLoader(pdf_path)

    print("Breaking Texts...")

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)


    chunks = loader.load_and_split(text_splitter)

    # --- HuggingFace Embeddings (FREE) ---
    print("Creating Vector Store... (Using HuggingFace Embeddings - 100% FREE)")
    embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    db = FAISS.from_documents(chunks, embedding=embeddings)

    print("--- PDF READY! --- Bot is ready to answer. ---")

    retriever = db.as_retriever()

    # --- Gemini Chat (FREE TIER) ---
    llm = ChatGoogleGenerativeAI(
        model="gemini-2.5-flash-preview-09-2025",
        temperature=0.3
    )

    print("Model connected: Gemini Chat (FREE) + HF Embeddings (FREE)")

    new_prompt_template = """
    You are a helpful assistant.

    First, try to answer the question using the 'Context' provided below.
    If the answer is not found in the context, it's okay, just use your general knowledge to answer the question.

    Context:
    {context}

    Question: {input}

    Answer:
    """

# Create the prompt object from the template
    prompt = ChatPromptTemplate.from_template(new_prompt_template)

    combine_docs_chain = create_stuff_documents_chain(llm, prompt)
    retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)

    print("\n🤖 Chatbot is LIVE! (HYBRID FREE Mode) (type 'exit' to stop the chatbot)")
    print("-" * 60)

    while True:
        user_question = input("What's your question? (Ask your question): ")
        if user_question.strip().lower() == 'exit':
            print("Bye Buddy! 👋")
            break

        print("Searching Answer... (Gemini is thinking...)")

        response = retrieval_chain.invoke({"input": user_question})

        print("\n--- Answer: ---")
        print(response.get("answer", "⚠️ No answer found. Try rephrasing your question."))
        print("-" * 60)

except Exception as e:
    print(f"\n--- ERROR! ------")
    print(traceback.format_exc())
    print("👉 Check: Google API Key is correct , 'quota' present ?, internet is on ?")

'syllabus.pdf' Reading...
Breaking Texts...
Creating Vector Store... (Using HuggingFace Embeddings - 100% FREE)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

--- PDF READY! --- Bot is ready to answer. ---
Model connected: Gemini Chat (FREE) + HF Embeddings (FREE)

🤖 Chatbot is LIVE! (HYBRID FREE Mode) (type 'exit' to stop the chatbot)
------------------------------------------------------------
What's your question? (Ask your question): what is object oriented programming
Searching Answer... (Gemini is thinking...)

--- Answer: ---
Object-Oriented Programming (OOP) is a programming paradigm based on the concept of "objects," which can contain both data (fields or attributes) and code (methods or procedures).

The primary goal of OOP is to organize software design around data, rather than logic and functions.

Key principles of Object-Oriented Programming include:
1.  **Encapsulation:** Bundling data and the methods that operate on that data into a single unit (the object), and restricting direct access to some of the object's components.
2.  **Inheritance:** The mechanism by which one class can acquire the properties and behavior of another